In [72]:
from dotenv import load_dotenv
load_dotenv()

import json

with open("user_cv.json",'r') as file:
    user = json.load(file)

def get_compliation(system_message, user_input, api_key=None):
    from openai import OpenAI

    if not api_key:
        api_key = os.environ['OPENAI_API_KEY']

    client = OpenAI(api_key=api_key)
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_input},
        ],
        stream=False,
    )
    return stream

In [73]:
print("Let's start with education")

education = sorted(user['education'],key=lambda x: x['start_date'])
education

Let's start with education


[{'degree': 'Full Matriculation',
  'institution': 'Kfar Ganim “Amit” high School',
  'location': '',
  'start_date': '2010',
  'graduation_date': '2013',
  'grade': ''},
 {'degree': 'Pre-Med and Biology',
  'institution': 'Tel Aviv University',
  'location': 'Tel Aviv',
  'start_date': '2020',
  'graduation_date': '2021',
  'grade': ''},
 {'degree': 'Computer science and Biology with specialisation in Bioinformatics',
  'institution': 'Tel Aviv University',
  'location': 'Tel Aviv',
  'start_date': '2022',
  'graduation_date': '2025',
  'grade': ''}]

In [74]:
prompt = f"""
I need you to interview me about this period in my life
{json.dumps(education[0],indent=4)}
You Goal is to asses what skills I've gained during this time.
What questions you will ask?

Response format:
```json
[
    {{
     "question": "the question to ask",
     "expection": "what you expected to learn from the user answer"
    }}
    // more if needed
]
```
"""
response = get_compliation("",prompt)

In [75]:
education_question = json.loads(response.choices[0].message.content.replace("```json","").replace("```",""))

In [76]:
education_question

[{'question': 'What subjects did you enjoy the most during your time at Kfar Ganim “Amit” high School?',
  'expectation': 'To learn about the subjects that you were passionate about and possibly excelled in, indicating strong interest and potential skills in those areas.'},
 {'question': 'Did you participate in any extracurricular activities or clubs while in high school? If so, which ones and what role did you have?',
  'expectation': 'To understand if you were involved in activities that may have developed teamwork, leadership, or other valuable skills outside of the traditional classroom setting.'},
 {'question': 'Can you recall a challenging project or assignment you had to complete during your high school years? How did you manage to overcome the challenge?',
  'expectation': 'To assess your problem-solving abilities, resilience, and perseverance in facing academic challenges.'},
 {'question': 'Did you take on any leadership roles in school, such as being a class representative or

In [78]:
for entry in education_question[:1]:
    print(f"Q: {entry['question']}")
    answer = input()
    entry['answer'] = answer


Q: What subjects did you enjoy the most during your time at Kfar Ganim “Amit” high School?


 ss


In [79]:
print("Let's start with experience")

experience = sorted(user['experience'],key=lambda x: x['start_date'])
experience

Let's start with experience


[{'title': 'Independent Psychometrics and Mathematics Tutor',
  'company': '',
  'location': '',
  'start_date': '2019',
  'end_date': '2020',
  'responsibilities': ['Designed personalized study plans and conducted effective teaching sessions to enhance student’s performance'],
  'keywords': []},
 {'title': 'Psychometric Essay Examiner',
  'company': 'The Psychometric of the State',
  'location': '',
  'start_date': '2020',
  'end_date': '2021',
  'responsibilities': ['Evaluated and assessed psychometric essays'],
  'keywords': []},
 {'title': 'Network Control Center',
  'company': 'Tel Aviv University',
  'location': 'Tel Aviv',
  'start_date': '2023',
  'end_date': '2024',
  'responsibilities': ['Monitoring network issues and serving as the primary contact for initial incident response (Troubleshooting)'],
  'keywords': []},
 {'title': 'Data Analyst & Engineering',
  'company': 'Tel Aviv University',
  'location': 'Tel Aviv',
  'start_date': '2024',
  'end_date': 'current',
  'respon

In [90]:

experience_test = experience[1]
system_prompt = f"""
You are interviewing me about this period in my life:
{json.dumps(experience_test,indent=4)}
You Goal is to asses what skills I've gained during this time.
When you think that you've understood the skill set I've gained you can respond with 'quit'.

Response format:
```json
{{
 "question": "the question to ask",
 "message": "the message to send the user, that contains the question. be nice!",
 "expectation": "what you expected to learn from the user answer"
}}
```
only one question at a time.
"""

In [91]:
experience_test['company']

'The Psychometric of the State'

In [93]:
from openai import OpenAI
import os
import json

# Load your API key from an environment variable or secret management service
api_key = os.environ['OPENAI_API_KEY']

def chatbot():
  print(f"Bot: Let's talk about your time at {experience_test['company']}")
  # Create a list to store all the messages for context
  messages = [
    {"role": "system", "content": system_prompt},
  ]

  # Keep repeating the following
  while True:

    # Request gpt-3.5-turbo for chat completion
    client = OpenAI(api_key=api_key)
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            stream = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            chat_message = json.loads(stream.choices[0].message.content.replace("```json","").replace("```",""))
            break  # Break out of the loop if successful
        except json.JSONDecodeError as e:
            retry_count += 1
            if retry_count == max_retries:
                raise e
            else:
                print("Bot: ....")
                

    # Print the response and add it to the messages list
    print(f"Bot: {chat_message['message']}")

    # Prompt user for input
    message = input("User: ")

    # Exit program if user inputs "quit"
    if message.lower() == "quit":
      break

    # Add each new message to the list
    messages.append({"role": "assistant", "content": json.dumps(chat_message,indent=4)})
    messages.append({"role": "user", "content": message})
    

if __name__ == "__main__":
  print("Start chatting with the bot (type 'quit' to stop)!")
  chatbot()

Start chatting with the bot (type 'quit' to stop)!
Bot: Let's talk about your time at The Psychometric of the State
Bot: It sounds like you gained experience in evaluating and assessing psychometric essays. Can you walk me through the process you followed when doing this?


User:   I don't know what do you think about that?


Bot: In assessing psychometric essays, what specific criteria did you use to evaluate them?


KeyboardInterrupt: Interrupted by user

In [66]:
projects = user['projects']
projects

[{'title': 'Raw Data Preparation using Pandas',
  'description': '',
  'technologies': ['Pandas'],
  'github_link': 'https://github.com/EhrlichNati/Garmin-Sports-Watch-Data-Cleanup'},
 {'title': 'CNN Classification From Scratch using Numpy',
  'description': '',
  'technologies': ['Numpy'],
  'github_link': 'https://github.com/EhrlichNati/CNN-from-scratch-Binary-Classification-'}]

In [56]:
projects[0]['github_link']

'https://github.com/EhrlichNati/Garmin-Sports-Watch-Data-Cleanup'

In [59]:
import requests
response = requests.get(projects[0]['github_link'])

AttributeError: 'Response' object has no attribute 'body'